<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">


# Web Scraping for Indeed.com and Predicting Salaries

### Business Case Overview

You're working as a data scientist for a contracting firm that's rapidly expanding. Now that they have their most valuable employee (you!), they need to leverage data to win more contracts. Your firm offers technology and scientific solutions and wants to be competitive in the hiring market. Your principal wants you to

   - determine the industry factors that are most important in predicting the salary amounts for these data.

To limit the scope, your principal has suggested that you *focus on data-related job postings*, e.g. data scientist, data analyst, research scientist, business intelligence, and any others you might think of. You may also want to decrease the scope by *limiting your search to a single region.*

Hint: Aggregators like [Indeed.com](https://www.indeed.com) regularly pool job postings from a variety of markets and industries.

**Goal:** Scrape your own data from a job aggregation tool like Indeed.com in order to collect the data to best answer this question.

---

### Directions

In this project you will be leveraging a variety of skills. The first will be to use the web-scraping and/or API techniques you've learned to collect data on data jobs from Indeed.com or another aggregator. Once you have collected and cleaned the data, you will use it to address the question above.

### Factors that impact salary

To predict salary the most appropriate approach would be a regression model.
Here instead we just want to estimate which factors (like location, job title, job level, industry sector) lead to high or low salary and work with a classification model. To do so, split the salary into two groups of high and low salary, for example by choosing the median salary as a threshold (in principle you could choose any single or multiple splitting points).

Use all the skills you have learned so far to build a predictive model.
Whatever you decide to use, the most important thing is to justify your choices and interpret your results. *Communication of your process is key.* Note that most listings **DO NOT** come with salary information. You'll need to be able to extrapolate or predict the expected salaries for these listings.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10").

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

The URL here has many query parameters:

- `q` for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- `l` for a location 
- `start` for what result number to start on

In [1]:
URL = "http://www.indeed.com/jobs?"

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook

pd.set_option('display.max_colwidth',1000)

Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is in a `span` with `class='salaryText'`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element='jobTitle'`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 
- Decide which other components could be relevant, for example the region or the summary of the job advert.

### Write 4 functions to extract each item: location, company, job, and salary.

Example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- **Make sure these functions are robust and can handle cases where the data/field may not be available.**
    - Remember to check if a field is empty or `None` for attempting to call methods on it.
    - Remember to use `try/except` if you anticipate errors.
- **Test** the functions on the results above and simple examples.

In [3]:
## YOUR CODE HERE

def extract_title_from_result(result):
    try:
        return result.find('a', attrs={'class':'jobtitle', 'data-tn-element':'jobTitle'}).text.strip()
    except:
        np.nan
    
def extract_company_from_result(result):
    try:
        return result.find('span', attrs={'class':'company'}).text.strip()
    except:
        return np.nan
        
def extract_rating_from_result(result):
    try:
        return result.find('span', attrs={'class':'ratingsDisplay'}).text.strip()
    except:
        return np.nan
        
def extract_location_from_result(result):
    try:
        return result.find('span', attrs={'class':'location'}).text.strip()
    except:
        return np.nan

def extract_summary_from_result(result):
    try:
        return result.find('div', attrs={'class':'summary'}).text.strip()
    except:
        return np.nan

def extract_date_from_result(result):
    try:
        return result.find('span', attrs={'class':'date'}).text.strip()
    except:
        return np.nan

def extract_salary_from_result(result):
    try:
        return result.find('span', attrs={'class':'salaryText'}).text.strip()
    except:
        return np.nan
    
def calc_salary_USD_from_salary(result):
    try:
        return extract_salary_from_result(result)*fx_rate[str(city)]
    except:
        return extract_salary_from_result(result)

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search.
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different.

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [19]:
## YOUR CODE HERE

import re

UK_CITY = ['London', 'Edinburgh', 'Cambridge', 'Oxford', 'Manchester', 'Reading', 'Bristol', 'Belfast',
            'Leeds', 'Glasgow', 'Birmingham', 'Nottingham']
UK_FX_rate = 1.3

max_results_per_city = 5000
job_city = {}
run_count = 0

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 'Los+Angeles', 'Philadelphia',
                 'Atlanta', 'Dallas', 'Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston',
                 'Miami'] + UK_CITY):
    
    title = []
    company = []
    rating = []
    search_city = []
    location = []
    salary = []
    age = []
    summary = []
    
    run_count+=1
    print('Searching City number: ', run_count)

    for occupation in set(['data scientist', 'data analyst', 'data analytics', 'data engineer',
                           'business intelligence', 'machine learning', 'artificial intelligence']): 
        old_page = 0
        print('Running: ', str(city),' & ',str(occupation))

        for page in tqdm_notebook(range(0, max_results_per_city, 10)):
            if city in UK_CITY:
                URL = "http://www.indeed.co.uk/jobs?"
                PARAMS = dict(as_phr=str(occupation), l=city, start=page)
            else:
                URL = "http://www.indeed.com/jobs?"
                PARAMS = dict(as_phr=str(occupation), l=city, start=page)

            r = requests.get(url=URL, params=PARAMS)
            soup = BeautifulSoup(r.text, 'html.parser')

            if soup.find('div', attrs={'id':'searchCountPages'}) == None:
                break
            else:
                new_page = int(re.findall(re.compile(r'\w+'), soup.find('div', attrs={'id':'searchCountPages'}).text.strip('\n'))[1])

                if new_page > old_page:
                    old_page = new_page
                    for posting in soup.find_all('div', attrs={'class':'result'}):
                        title.append(extract_title_from_result(posting))
                        company.append(extract_company_from_result(posting))
                        rating.append(extract_rating_from_result(posting))
                        search_city.append(city)
                        location.append(extract_location_from_result(posting))
                        salary.append(extract_salary_from_result(posting))
                        age.append(extract_date_from_result(posting))
                        summary.append(extract_summary_from_result(posting))
                else:
                    break

    # creating a df for that city's results and writing it to [insert city].csv...
    job_city_df = pd.DataFrame(dict(title=title,company=company,rating=rating,search_city=search_city,
                                    location=location,salary=salary,age=age,summary=summary))
    job_city_df.to_csv(str(city)+'.csv', index=False)
    
print('FINISHED')

Running:  Austin  &  artificial intelligence


Running:  Austin  &  data analyst


Running:  Austin  &  business intelligence


Running:  Austin  &  data scientist


Running:  Austin  &  data analytics


...

Running:  Birmingham  &  data scientist


Running:  Birmingham  &  data analytics


Running:  Birmingham  &  machine learning


Running:  Birmingham  &  data engineer


FINISHED


In [34]:
# Load in CSVs and create a dataframe from it:

jobs = pd.DataFrame()

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 'Los+Angeles', 'Philadelphia',
                 'Atlanta', 'Dallas', 'Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston',
                 'Miami'] + UK_CITY):

    jobs_city = pd.read_csv(str(city)+'.csv')

    if 'Unnamed: 0' in pd.read_csv(str(city)+'.csv').columns: # to check if csv was saved with index (above)
        jobs_city.drop(columns=['Unnamed: 0'], inplace=True)
    else:
        break
    jobs = jobs.append(jobs_city, ignore_index=True)

print(jobs.shape)
jobs.head()

(49527, 8)


,title,company,rating,search_city,location,salary,age,summary
0,Artificial Intelligence/Machine Learning Engineer,BPMLinks,NaN,Denver,"Denver, CO",NaN,30+ days ago,Strong background in statistical concepts and calculations.Developing AI and ML models.Researching on new AI and ML techniques.
1,Anti-Money Laundering Analyst,IBM,3.9,Denver,"Denver, CO 80221",NaN,Just posted,"By delivering artificial intelligence, analytics, security, blockchain and cloud technology as a service, and in a complete platform tailored to the needs of…"
2,account manager,BusinesStaff,NaN,Denver,"Denver, CO","$170,000 - $220,000 a year",30+ days ago,"Strategic mindset with proven ability to synthesize customer financial reports, industry information and market intelligence to develop customer growth…"
3,"Artificial Intelligence Manager, CMT Southwest",Accenture,4.0,Denver,"Denver, CO",NaN,30+ days ago,"Partnering with our client to drive overall technology transformation agenda.Driving the methods, strategies and approach to Cloud adoption."
4,Anti-Money Laundering Team Lead,IBM,3.9,Denver,"Denver, CO 80221",NaN,Just posted,"By delivering artificial intelligence, analytics, security, blockchain and cloud technology as a service, and in a complete platform tailored to the needs of…"


Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now.
1. Some of the entries may be duplicated.
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries.

In [35]:
## YOUR CODE HERE: adding salary frequencies, and restricting on yearly salaries, also dropping dupes.
jobs_salaried = jobs[jobs['salary'].notnull()]

# find and remove dupes first:
print('Total shape:',jobs_salaried.shape)
print('Duplicates:',jobs_salaried[jobs_salaried.duplicated()].shape)

jobs_salaried.drop_duplicates(inplace=True)
print('Any dupes left?:',jobs_salaried[jobs_salaried.duplicated()].shape)

# create salary_freq column and filter for only yearly salaries:
salary_freq = []
for sal in jobs_salaried['salary']:
    if 'year' in sal:
        salary_freq.append('year')
    elif 'month' in sal:
        salary_freq.append('month')
    elif 'week' in sal:
        salary_freq.append('week')
    elif 'day' in sal:
        salary_freq.append('day')
    elif 'hour' in sal:
        salary_freq.append('hour')
    else:
        salary_freq.append(np.nan)
        
jobs_salaried.insert(6, 'salary_freq', salary_freq)
print('Unique Salary Frequencies found:',jobs_salaried.salary_freq.unique())

jobs_salaried_year = jobs_salaried[jobs_salaried['salary_freq']=='year']
print('Checking only year remains:',jobs_salaried_year.salary_freq.unique())
print('Jobs Salaried Yearly:',jobs_salaried_year.shape)

Total shape: (7264, 8)
Duplicates: (3570, 8)
Any dupes left?: (0, 8)
Unique Salary Frequencies found: ['year' 'hour' 'month' nan 'day' 'week']
Checking only year remains: ['year']
Jobs Salaried Yearly: (2999, 9)


/Users/Noah/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary.

In [36]:
## YOUR CODE HERE: creating salary_avg from salary ranges, and dollarizing UK salaries

import re
from re import sub
from decimal import Decimal

jobs_salaried_year.insert(7, 'isRange', jobs_salaried_year.apply(lambda x: 'Range' if '-' in x['salary'] else
                                                        'Not Range', axis=1))

salary_range_min = []
salary_range_max = []
salary_avg = []
salary_avg_USD = []

for x in jobs_salaried_year[['salary','isRange']].values:
    if x[1] == 'Not Range':
        salary_range_min.append(Decimal(sub(r'[^\d.]', '', x[0])))
        salary_range_max.append(Decimal(sub(r'[^\d.]', '', x[0])))
        salary_avg.append(Decimal(sub(r'[^\d.]', '', x[0])))

    if x[1] == 'Range':
        range_min = re.findall(r'[\d.,]+',x[0])[0]
        range_min = Decimal(sub(r',','',range_min))
        salary_range_min.append(range_min)
        
        range_max = re.findall(r'[\d.,]+',x[0])[1]
        range_max = Decimal(sub(r',','',range_max))
        salary_range_max.append(range_max)
        
        salary_avg.append((range_min+range_max)/2)
    
jobs_salaried_year.insert(8, 'salary_range_min', salary_range_min)
jobs_salaried_year.insert(9, 'salary_range_max', salary_range_max)
jobs_salaried_year.insert(10, 'salary_avg', salary_avg)

jobs_salaried_year.insert(11, 'salary_avg_USD',
                         jobs_salaried_year.apply(lambda x: round(Decimal(UK_FX_rate)*x['salary_avg'],2) 
                                                  if x['search_city'] in UK_CITY
                                                  else x['salary_avg'], axis=1))

jobs_salaried_year.head(1)

,title,company,rating,search_city,location,salary,salary_freq,isRange,salary_range_min,salary_range_max,salary_avg,salary_avg_USD,age,summary
2,account manager,BusinesStaff,NaN,Denver,"Denver, CO","$170,000 - $220,000 a year",year,Range,170000,220000,195000,195000,30+ days ago,"Strategic mindset with proven ability to synthesize customer financial reports, industry information and market intelligence to develop customer growth…"


### Save your results as a CSV

In [37]:
## YOUR CODE HERE: write to csv
jobs_salaried_year.to_csv('indeed_results.csv', index=False)

### Load in the the data of scraped salaries

In [3]:
## YOUR CODE HERE
indeed = pd.read_csv('indeed_results.csv')

### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median).

We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't have to choose the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries.

In [4]:
## YOUR CODE HERE: categorise salary into 0-25%-ile, 25-50%-ile, 50-75%-ile, 75-100%-ile

def categorise(x):
    if x <= indeed.salary_avg_USD.quantile(0.25):
        return '0_25'
    elif x > indeed.salary_avg_USD.quantile(0.25) and x <= indeed.salary_avg_USD.quantile(0.5):
        return '25_50'
    elif x > indeed.salary_avg_USD.quantile(0.5) and x <= indeed.salary_avg_USD.quantile(0.75):
        return '50_75'
    elif x > indeed.salary_avg_USD.quantile(0.75):
        return '75_100'
    else:
        return np.nan

indeed.insert(12, 'target', indeed['salary_avg_USD'].apply(categorise))

#### Thought experiment: What is the baseline accuracy for this model?

In [5]:
## YOUR CODE HERE: baseline accuracy
baseline = indeed.target.value_counts(normalize=True).max()
print('Baseline Accuracy:',baseline)
print('Normalized Value Counts:')
print(indeed.target.value_counts(normalize=True))

Baseline Accuracy: 0.25608536178726243
Normalized Value Counts:
25_50     0.256085
0_25      0.251751
50_75     0.247416
75_100    0.244748
Name: target, dtype: float64


### Create a classification model to predict High/Low salary. 


- Start by ONLY using the location as a feature.
- Use at least two different classifiers you find suitable.
- Remember that scaling your features might be necessary.
- Display the coefficients/feature importances and write a short summary of what they mean.
- Create a few new variables in your dataframe to represent interesting features of a job title (e.g. whether 'Senior' or 'Manager' is in the title).
- Incorporate other text features from the title or summary that you believe will predict the salary.
- Then build new classification models including also those features. Do they add any value?
- Tune your models by testing parameter ranges, regularization strengths, etc. Discuss how that affects your models.
- Discuss model coefficients or feature importances as applicable.

In [9]:
## YOUR CODE HERE: using two different classifiers on location only

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier

# set X,y
X = indeed['search_city']
y = indeed['target']

# dummify X
X_dum = pd.get_dummies(X, drop_first=True)

# scale
scaler = StandardScaler()
Xs = scaler.fit_transform(X_dum)

# DecisionTreeClassifier:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
dt = DecisionTreeClassifier()
dt.fit(Xs, y)
print('Best Training CVScore:', cross_val_score(dt, Xs, y, cv=5).mean())

# GridSearchCV with Bagging (and Decision Tree base_estimator):
bc = BaggingClassifier(n_estimators=100, n_jobs=2)
bc_params = {'max_features': np.linspace(0.1, 1, 5)}

gs_bc = GridSearchCV(bc, bc_params, cv=kf, n_jobs=2, verbose=1)
gs_bc.fit(Xs,y)
print('Best GS BC params:', gs_bc.best_params_)
best_gs_bc = gs_bc.best_estimator_
print('Best Training GS BC CVScore:', gs_bc.best_score_)

Best Training CVScore: 0.26644129104062325
Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Best GS BC params: {'max_features': 0.775}
Best Training GS BC CVScore: 0.3434462993878687


[Parallel(n_jobs=2)]: Done  25 out of  25 | elapsed:    7.5s finished


In [22]:
# Summarise Feature Importance:
dt_feature_importance = pd.DataFrame({'importance':dt.feature_importances_}, index=X_dum.columns)
dt_feature_importance.sort_values('importance',ascending=False).T

# we see that Birmingham is the most important location feature, which means that it was both used often in the
# Decision Tree Classifier and when it was used yielded a significant improvement in the gini

,Birmingham,Nottingham,Reading,Cambridge,Edinburgh,London,Miami,San+Francisco,Pittsburgh,Chicago,Dallas,Phoenix,Houston,Portland,Austin,Denver,Los+Angeles,New+York,Seattle,Philadelphia
importance,0.164971,0.142649,0.139754,0.127168,0.109553,0.079659,0.047117,0.039418,0.022913,0.021399,0.018418,0.017877,0.017232,0.013029,0.012889,0.009878,0.008751,0.004295,0.002027,0.001004


In [20]:
# Summarize accuracy scores:

predictions = best_gs_bc.predict(Xs)
print('Confusion Matrix:')
print(confusion_matrix(y, predictions))
print(' ')
print('Classification Report:')
print(classification_report(y, predictions))

# the best classifier model at this stage is the BaggingClassifier, we see that recall is high 50_75 leading to
# low recall for the other classes, and also low precision for it's own class.

Confusion Matrix:
[[296  60 360  39]
 [200 101 390  77]
 [129  43 456 114]
 [ 57  20 443 214]]
 
Classification Report:
              precision    recall  f1-score   support

        0_25       0.43      0.39      0.41       755
       25_50       0.45      0.13      0.20       768
       50_75       0.28      0.61      0.38       742
      75_100       0.48      0.29      0.36       734

    accuracy                           0.36      2999
   macro avg       0.41      0.36      0.34      2999
weighted avg       0.41      0.36      0.34      2999



In [40]:
# Feature-engineering Title column:
# we start with TfidfVectorizer to get the unique features, and fit to regression to assess importance

from sklearn.feature_extraction.text import TfidfVectorizer

tvec = TfidfVectorizer(stop_words='english', token_pattern='[A-Za-z]+', ngram_range=(1,1))
tvec.fit(indeed['title'])

X = tvec.transform(indeed['title'])
y = indeed.target

lr = LogisticRegression(solver='lbfgs', multi_class='ovr')
print(cross_val_score(lr,X,y,cv=kf).mean())
lr.fit(X,y)
pd.DataFrame(dict(coef=lr.coef_[0], features=tvec.get_feature_names())).sort_values('coef',ascending=False).head(5)

0.5315225375626043


,coef,features
485,4.502704,graduate
602,3.166491,junior
57,3.123792,apprentice
924,3.017499,recruitment
396,2.717922,executive


In [39]:
pd.DataFrame(dict(coef=lr.coef_[0], features=tvec.get_feature_names())).sort_values('coef',ascending=True).head()

,coef,features
302,-2.802431,director
624,-2.620541,lead
363,-2.275998,engineer
1019,-1.977597,senior
1004,-1.888458,scientist


In [38]:
# We choose the below interesting features to engineer for:

indeed.insert(1, 'isDS', indeed['title'].apply(lambda x: 1 if 'data science' in x.lower() or 'data scientist' in x.lower() else 0))
indeed.insert(2, 'isDA', indeed['title'].apply(lambda x: 1 if 'data analyst' in x.lower() or 'analytics' in x.lower() else 0))
indeed.insert(3, 'isDE', indeed['title'].apply(lambda x: 1 if 'data engineer' in x.lower() else 0))
indeed.insert(4, 'isML', indeed['title'].apply(lambda x: 1 if 'ml' in x.lower() or 'machine learning' in x.lower() else 0))
indeed.insert(5, 'isAI', indeed['title'].apply(lambda x: 1 if 'ai' in x.lower() or 'artifical intelligence' in x.lower() else 0))
indeed.insert(6, 'isBI', indeed['title'].apply(lambda x: 1 if 'bi' in x.lower() or 'business intelligence' in x.lower() else 0))

indeed.insert(7, 'isDirector', indeed['title'].apply(lambda x: 1 if 'director' in x.lower() else 0))
indeed.insert(8, 'isHead', indeed['title'].apply(lambda x: 1 if 'head' in x.lower() else 0))
indeed.insert(9, 'isLead', indeed['title'].apply(lambda x: 1 if 'lead' in x.lower() else 0))
indeed.insert(10, 'isManager', indeed['title'].apply(lambda x: 1 if 'mgr' in x.lower() or 'manager' in x.lower() else 0))
indeed.insert(11, 'isSenior', indeed['title'].apply(lambda x: 1 if 'sr' in x.lower() or 'senior' in x.lower() else 0))
indeed.insert(12, 'isJunior', indeed['title'].apply(lambda x: 1 if 'jr' in x.lower() or 'junior' in x.lower() else 0))
indeed.insert(13, 'isGraduate', indeed['title'].apply(lambda x: 1 if 'grad' in x.lower() or 'graduate' in x.lower() else 0))
indeed.insert(14, 'isApprentice', indeed['title'].apply(lambda x: 1 if 'apprentice' in x.lower() else 0))

indeed.head(1)


,title,isDS,isDA,isDE,isML,isAI,isBI,isDirector,isHead,isLead,...,salary,salary_freq,isRange,salary_range_min,salary_range_max,salary_avg,salary_avg_USD,target,age,summary
0,account manager,0,0,0,0,0,0,0,0,0,...,"$170,000 - $220,000 a year",year,Range,170000,220000,195000.0,195000.0,75_100,30+ days ago,"Strategic mindset with proven ability to synthesize customer financial reports, industry information and market intelligence to develop customer growth…"


In [41]:
indeed.columns

Index(['title', 'isDS', 'isDA', 'isDE', 'isML', 'isAI', 'isBI', 'isDirector',
       'isHead', 'isLead', 'isManager', 'isSenior', 'isJunior', 'isGraduate',
       'isApprentice', 'company', 'rating', 'search_city', 'location',
       'salary', 'salary_freq', 'isRange', 'salary_range_min',
       'salary_range_max', 'salary_avg', 'salary_avg_USD', 'target', 'age',
       'summary'],
      dtype='object')

In [45]:
# DecisionTreeClassifier with new features added to assess improvement (not the BaggingClassifier as I want to 
# be able to easily pull out feature importances):

X_dum_title = pd.concat([X_dum,indeed.loc[:,'isDS':'isApprentice']], axis=1)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
dt = DecisionTreeClassifier()
dt.fit(X_dum_title, y)
print('Best Training CVScore:', cross_val_score(dt, X_dum_title, y, cv=5).mean())

# shows improvement vs. the previous DecisionTreeClassifier, let's display which features most importantly:

dt_feature_importance = pd.DataFrame({'importance':dt.feature_importances_}, index=X_dum_title.columns)
dt_feature_importance.sort_values('importance',ascending=False).T

Best Training CVScore: 0.3641062882582081


,isDA,isManager,isGraduate,isBI,Birmingham,Nottingham,isDS,isDE,isSenior,isLead,...,Los+Angeles,Pittsburgh,Denver,Miami,Portland,Phoenix,Chicago,Philadelphia,Seattle,Houston
importance,0.071094,0.069634,0.067182,0.051295,0.045384,0.044083,0.042015,0.040493,0.039957,0.039591,...,0.012076,0.011097,0.010096,0.00906,0.008778,0.008673,0.007837,0.007095,0.005501,0.002909


In [ ]:
# Incorporate other text features from the title or summary that you believe will predict the salary:
    # Do they add any value?
tvec = TfidfVectorizer(stop_words='english', token_pattern='[A-Za-z]+', ngram_range=(1,1))
tvec.fit(indeed['summary'])

tvec_summary = pd.DataFrame(tvec.transform(indeed['summary']).toarray(), columns=tvec.get_feature_names())

X_summary = X_dum_title.loc[:,'Austin':'Seattle'].add_prefix('is') # prefix pre-existing cols to avoid dupe from NLP
X_summary = pd.concat([X_summary,tvec_summary], axis=1)

# scaler = StandardScaler()
# Xs_summary = scaler.fit_transform(X_summary)

dt = DecisionTreeClassifier()
dt.fit(X_summary, y)
print('Best Training CVScore:', cross_val_score(dt, X_summary, y, cv=5).mean())

dt_feature_importance = pd.DataFrame({'importance':dt.feature_importances_}, index=X_summary.columns)
dt_feature_importance.sort_values('importance',ascending=False).T

# Need to make pipeline with the steps needed, and also a loop through the models to test, all within in a
# gridsearch! Let's build that up!

# Ask Chris too if cross val deterioration is expected?

### Model evaluation:

Your boss would rather tell a client incorrectly that they would get a lower salary job than tell a client incorrectly that they would get a high salary job. Adjust one of your models to ease his mind, and explain what it is doing and any tradeoffs.


- Use cross-validation to evaluate your models.
- Evaluate the accuracy, AUC, precision and recall of the models.
- Plot the ROC and precision-recall curves for at least one of your models.

In [ ]:
## YOUR CODE HERE

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### Bonus:

- Answer the salary discussion by using your model to explain the tradeoffs between detecting high vs low salary positions. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.
- Obtain the ROC/precision-recall curves for the different models you studied (at least the tuned model of each category) and compare.

In [ ]:
## YOUR CODE HERE

### Summarize your results in an executive summary written for a non-technical audience.
   
- Writeups should be at least 500-1000 words, defining any technical terms, explaining your approach, as well as any risks and limitations.

In [ ]:
## YOUR TEXT HERE IN MARKDOWN FORMAT 

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### BONUS

Convert your executive summary into a public blog post of at least 500 words, in which you document your approach in a tutorial for other aspiring data scientists. Link to this in your notebook.

In [ ]:
## YOUR LINK HERE IN MARKDOWN FORMAT 